In [2]:
import os
import pandas as pd
import glob 

In [ ]:
import scipy
import numpy as np
import scipy.io as spio


def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [ ]:
homedir='/media/raghuram/My Passport/dicom_seg/TCGA-LGG'

In [ ]:
os.chdir(homedir)

In [ ]:
dicom_file_params_df=pd.read_csv('dicom_file_params.csv')

In [ ]:
dicom_file_params_df.shape

In [ ]:
t1_post_samples_df = pd.read_csv('t1_post_samples.csv')
t1_pre_samples_df = pd.read_csv('t1_pre_samples.csv')
flair_samples = pd.read_csv('flair_samples.csv')
t2_pre_samples = pd.read_csv('t2_pre_samples.csv')

In [ ]:
# t1_post_samples_df['Sequence Name']

In [ ]:
sequence_mapper_dict = {'T1CE':t1_post_samples_df.values,
               'T1W':t1_pre_samples_df.values,
               'T2W': t2_pre_samples.values,
               'T2F': flair_samples.values}

In [ ]:
mapped_sequence = []
for idx, row in dicom_file_params_df.iterrows():
    try:
        if row['scanning_seq_mri'] in sequence_mapper_dict['T1CE']:
            mapped_sequence.append('T1CE')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T1W']:
            mapped_sequence.append('T1W')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T2W']:
            mapped_sequence.append('T2W')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T2F']:
            mapped_sequence.append('T2F')
    except Exception as e:
        print('{}, {} in row {}'.format(e, row['scanning_seq_mri'], idx))

In [ ]:
dicom_file_params_df['mat_file_sequence'] = mapped_sequence
dicom_file_params_df.to_csv('dicom_file_params.csv', index=False)

In [ ]:
patient_name_list = []
for idx, row in dicom_file_params_df.iterrows():
    try:
        patient_name_list.append(row['filename'].split('/TCGA-LGG')[1].split('/')[1])
    except Exception as e:
        print('Error {} at index {}'.format(e, idx))

In [ ]:
dicom_file_params_df['patient_name'] = patient_name_list
dicom_file_params_df.to_csv('dicom_file_params.csv', index=False)

In [ ]:
mat_files_dir = '/home/raghuram/Desktop/radiomics/TEXTURES'

In [ ]:
os.chdir(mat_files_dir)

In [ ]:
mat_files_list = glob.glob('*.mat')

In [ ]:
data = loadmat(mat_files_list[0])

In [ ]:
mat_files_list[-1 ]

In [ ]:
dicom_file_params_df['mat_file_name'] = dicom_file_params_df['patient_name']+'_'+dicom_file_params_df['mat_file_sequence']+'.mat'

In [ ]:
dicom_file_params_df.drop_duplicates(subset=['mat_file_name'], keep='first', inplace=True)

In [ ]:
dicom_file_params_df.drop(columns=['patient_name'], inplace=True)

In [ ]:
dicom_file_params_df.to_csv('dicom_file_params_df_with_mat_files.csv', index=False)

In [ ]:
missing = set(mat_files_list).difference(list(dicom_file_params_df['mat_file_name']))

In [ ]:
# missing

In [ ]:
mat_files_list[0]

In [ ]:
dicom_file_params_df.head()

In [ ]:
def extract_flatten_features(mat_data, filename):
    
    features_flattened = []
    
    for experiment_, values in data['textures']['List'].items():
            
        experiment_number = int(experiment_.split('Experiment')[1])
        if experiment_number > 25:
            break
        scale_ = float(values.split(',')[0].split('=')[1])
        algo_ = values.split(',')[1].split('=')[1]
        ng_ = int(values.split(',')[2].split('=')[1])
        
        flattened_df = pd.io.json.json_normalize(data['textures'][experiment_], sep='_')
        flattened_df['mat_file_name'] = filename
        flattened_df_merged = pd.merge(flattened_df, dicom_file_params_df, on='mat_file_name', how='inner')
        flattened_df_merged['experiment_number'] = experiment_number
        flattened_df_merged['scale'] = scale_
        flattened_df_merged['algo'] = algo_
        flattened_df_merged['ng'] = ng_
        features_flattened.append(flattened_df_merged)
    
    features_df_concat = pd.concat(features_flattened, ignore_index=True)
    features_df_concat.to_csv(filename.split('.')[0]+'_features'+'.csv', index=False)

In [ ]:
# extract_flatten_features(data)

In [ ]:
for idx, mat_file in enumerate(mat_files_list):
    
    if mat_file in missing:
            print('Filename not in dicom df, skipping')
            continue   
    
    print('Mat file {} at index {} of total length {} being processed'.format(mat_file, idx, len(mat_files_list)))
    data = loadmat(mat_file)
    extract_flatten_features(data, mat_file)

In [3]:
homedir = '/home/raghuram/Desktop/radiomics/TEXTURES/csv_folder'
os.chdir(homedir)

t1_list = glob.glob('*_T1W_features.csv')
t2_list = glob.glob('*_T2W_features.csv')
t1ce_list = glob.glob('*_T1CE_features.csv')
flair_list = glob.glob('*_T2F_features.csv')

In [57]:
def create_npy_files(sequence_list, sequence_name):
    
    for expt_num in range(1, 26):
        df_list = []
        for seq_file in t1_list:
            df = pd.read_csv(seq_file)
            df = df[df['experiment_number'] == expt_num]
            df_list.append(df)
        pd.concat(df_list).to_csv('seq_file'+'_'+str(expt_num)+'_'+sequence_name+'.csv')

In [62]:
create_npy_files(flair_list, 'T2F')

In [25]:
df = pd.read_csv(t1_list[0])

In [26]:
df.head()

,GLCM_Contrast,GLCM_Correlation,GLCM_Dissimilarity,GLCM_Energy,GLCM_Entropy,GLCM_Homogeneity,GLCM_SumAverage,GLCM_Variance,GLRLM_GLN,GLRLM_GLV,...,mag_field_strength,scanner_manufacturer,scanner_model,repetition_time,excitation_time,mat_file_sequence,experiment_number,scale,algo,ng
0,0.624561,0.939791,0.448921,0.057985,4.492363,0.799303,0.069671,0.081040,0.135238,0.001243,...,3.0,GE MEDICAL SYSTEMS,SIGNA EXCITE,6.773,2.772,T1W,1,0.5,Equal,8
1,2.104891,0.949642,0.914987,0.018196,6.264325,0.667598,0.032877,0.081637,0.066386,0.002978,...,3.0,GE MEDICAL SYSTEMS,SIGNA EXCITE,6.773,2.772,T1W,2,0.5,Equal,16
2,7.951513,0.952384,1.838260,0.005276,8.141398,0.522198,0.015953,0.081539,0.032823,0.004549,...,3.0,GE MEDICAL SYSTEMS,SIGNA EXCITE,6.773,2.772,T1W,3,0.5,Equal,32
3,28.249120,0.953508,3.519248,0.001526,10.000019,0.389452,0.007621,0.074171,0.016778,0.006702,...,3.0,GE MEDICAL SYSTEMS,SIGNA EXCITE,6.773,2.772,T1W,4,0.5,Equal,64
4,0.149167,0.871195,0.147316,0.281439,2.349336,0.926647,0.068091,0.009047,0.309494,0.000549,...,3.0,GE MEDICAL SYSTEMS,SIGNA EXCITE,6.773,2.772,T1W,5,0.5,Uniform,8
